# Анализ пользовательских данных и заказов e-commerce платформы - SQL Код

<img src="https://storage.yandexcloud.net/klms-public/production/learning-content/152/1762/17923/51794/244290/2023_01_24_214337_negate.jpg" alt="Alt text" width="600">


### 1. Подсчитайте количество сделанных и отмененных заказов для каждого месяца.

```sql
SELECT date_trunc('month', time) as month,
       action,
       count(order_id) as orders_count
FROM   user_actions
GROUP BY month, action
ORDER BY month, action
```

### 2. Подсчитайте количество пользователей в каждой подгруппе: возрастные группы и пол.

```sql
SELECT date_part('year', age(birth_date))::integer as age,
       sex,
       count(user_id) as users_count
FROM   users
WHERE  birth_date is not null
GROUP BY age, sex
ORDER BY age, sex
```

### 3. Сгруппируйте данные по количеству товаров в заказах, учитывая только заказы, оформленные по будням, и посчитайте количество заказов в каждой группе. Включите в результат только те группы, где количество заказов превышает 2000.


```sql
SELECT array_length(product_ids, 1) as order_size,
       count(order_id) as orders_count
FROM   orders
WHERE  to_char(creation_time, 'Dy') NOT IN ('Sat', 'Sun')
GROUP BY order_size having count(order_id) > 2000
ORDER BY order_size
```

### 4. Сгруппируйте пользователей по четырём возрастным категориям: «18-24», «25-29», «30-35», «36+», и посчитайте количество пользователей в каждой группе. Исключите из расчётов пользователей с неуказанной датой рождения, учитывая полные года.

```sql
SELECT case when date_part('year',
                           age(current_date, birth_date))::integer between 18 and
                 24 then '18-24'
            when date_part('year', age(current_date, birth_date))::integer between 25 and
                 29 then '25-29'
            when date_part('year', age(current_date, birth_date))::integer between 30 and
                 35 then '30-35'
            else '36+' end as group_age,
       count(user_id) as users_count
FROM   users
WHERE  birth_date IS NOT null
GROUP BY group_age
ORDER BY group_age
```

### 5.  Рассчитайте средний размер заказа по выходным и будним дням.

```sql
SELECT case when date_part('dow', creation_time) between 2 and 6 then 'weekdays'
            else 'weekend' end as week_part,
       round(avg(array_length(product_ids, 1)), 2) as avg_order_size
FROM   orders
GROUP BY week_part
ORDER BY avg_order_size
```

### 6. Посчитайте общее количество оформленных заказов и долю отменённых заказов для каждого пользователя. Включите в результат только тех пользователей, которые оформили более трёх заказов и имеют показатель cancel_rate не менее 0.5.

```sql
SELECT user_id,
       round(
       count(distinct order_id) filter (WHERE action = 'cancel_order')::decimal
       / count(distinct order_id),
       2) as cancel_rate,
       count(distinct order_id) as orders_count
FROM   user_actions
GROUP BY user_id having round(count(distinct order_id) filter (
WHERE  action = 'cancel_order')::decimal / count(distinct order_id), 2) >= 0.5
       and count(distinct order_id) > 3
ORDER BY user_id
```

### 7. Для каждого дня недели в таблице `user_actions` за период с 24 августа по 6 сентября 2022 года посчитайте:

- Общее количество оформленных заказов (created_orders).
- Общее количество отменённых заказов (canceled_orders).
- Общее количество неотменённых заказов (actual_orders).
- Долю неотменённых заказов (success_rate), округлив её до трёх знаков после запятой.

```sql
SELECT date_part('isodow', time)::int as weekday_number,
       to_char(time, 'Dy') as weekday,
       count(order_id) filter (WHERE action = 'create_order') as created_orders,
       count(order_id) filter (WHERE action = 'cancel_order') as canceled_orders,
       count(order_id) filter (WHERE action = 'create_order')
           - count(order_id) filter (WHERE action = 'cancel_order') as actual_orders,
       round(
           (count(order_id) filter (WHERE action = 'create_order')
            - count(order_id) filter (WHERE action = 'cancel_order'))::decimal
           / count(order_id) filter (WHERE action = 'create_order'),
       3) as success_rate
FROM   user_actions
WHERE  time >= '2022-08-24'
       and time < '2022-09-07'
GROUP BY weekday_number, weekday
ORDER BY weekday_number
```

### 8. Oпределите возраст самого молодого курьера мужского пола в таблице `couriers`, в качестве первой даты используйте последнюю дату из таблицы `courier_actions`.

```sql
SELECT min(age((SELECT max(time)::date
                FROM   courier_actions), birth_date))::varchar as min_age
FROM   couriers
WHERE  sex = 'male'
```

### 9. Рассчитайте количество заказов для каждого пользователя и отразите это в столбце orders_count. В отдельном столбце orders_avg укажите среднее число заказов всех пользователей, округлив его до двух знаков после запятой.


```sql
with t1 as (SELECT user_id,
                   count(order_id) as orders_count
            FROM   user_actions
            WHERE  action = 'create_order'
            GROUP BY user_id)
SELECT user_id,
       orders_count,
       round((SELECT avg(orders_count)
       FROM   t1), 2) as orders_avg, orders_count - round((SELECT avg(orders_count)
                                                    FROM   t1), 2) as orders_diff
FROM   t1
ORDER BY user_id
```

### 10. Определите число недоставленных заказов и среди них посчитайте количество отменённых заказов и количество заказов, которые не были отменены (и соответственно, пока ещё не были доставлены).

```sql
SELECT count(distinct order_id) as orders_undelivered,
       count(order_id) filter (WHERE action = 'cancel_order') as orders_canceled,
       count(distinct order_id) - count(order_id) filter (WHERE action = 'cancel_order') as orders_in_process
FROM   user_actions
WHERE  order_id in (SELECT order_id
                    FROM   courier_actions
                    WHERE  order_id not in (SELECT order_id
                                            FROM   courier_actions
                                            WHERE  action = 'deliver_order'))
```

### 11. Выведите id и содержимое 100 последних доставленных заказов из таблицы orders. Содержимым заказов считаются списки с id входящих в заказ товаров. 

```sql
SELECT order_id,
       product_ids
FROM   orders
WHERE  order_id IN (SELECT order_id
                    FROM   courier_actions
                    WHERE  action = 'deliver_order'
                    ORDER BY time DESC
                    LIMIT 100)
ORDER BY order_id
```

### 12. Из таблицы `couriers` выведите всю информацию о курьерах, которые в сентябре 2022 года доставили 30 и более заказов.



```sql
SELECT courier_id,
       birth_date,
       sex
FROM   couriers
WHERE  courier_id in (SELECT courier_id
                      FROM   courier_actions
                      WHERE  action = 'deliver_order'
                             and date_part('year', time) = 2022
                             and date_part('month', time) = 9
                      GROUP BY courier_id
                      HAVING COUNT(distinct order_id) >= 30)
ORDER BY courier_id
```

### 13. Рассчитайте средний размер заказов, отменённых пользователями мужского пола. Средний размер заказа округлите до трёх знаков после запятой. 

```sql
SELECT round(avg(array_length(product_ids, 1)), 3) as avg_order_size
FROM   orders
WHERE  order_id in (SELECT order_id
                    FROM   user_actions
                    WHERE  action = 'cancel_order'
                       and user_id in (SELECT user_id
                                    FROM   users
                                    WHERE  sex = 'male'))
```

### 14. Посчитайте возраст каждого пользователя в таблице `users` в полных годах относительно последней даты в таблице `user_actions`. Для пользователей без указанной даты рождения укажите средний возраст всех остальных пользователей, округлённый до целого числа.


```sql
WITH info as (
    SELECT  user_id,
            DATE_PART('year',
            age(
            (SELECT max(time)
            from user_actions),
            birth_date)) as age
    FROM users
)
SELECT  user_id, 
        coalesce(age,
        (SELECT AVG(age)
        FROM info))::integer as age
FROM info
ORDER BY user_id
```

### 15. Для каждого неотменённого заказа, в котором больше 5 товаров, рассчитайте время, затраченное на его доставку. Включите в результат id заказа, время принятия заказа курьером (time_accepted), время доставки заказа (time_delivered), и время, затраченное на доставку в минутах, округлив его до целого числа (delivery_time). 


```sql
SELECT order_id,
       min(time) as time_accepted,
       max(time) as time_delivered,
       (extract(epoch
FROM   max(time) - min(time))/60)::integer as delivery_time
FROM   courier_actions
WHERE  order_id in (SELECT order_id
                    FROM   orders
                    WHERE  array_length(product_ids, 1) > 5)
       and order_id not in (SELECT order_id
                            FROM   user_actions
                            WHERE  action = 'cancel_order')
GROUP BY order_id
ORDER BY order_id
```

### 16. Для каждой даты в таблице user_actions посчитайте количество первых заказов, совершённых пользователями, которые сделали заказ впервые. Учитывайте только неотменённые заказы.


```sql
WITH info as (SELECT *,
                     date(time) as date
              FROM   user_actions)
SELECT date,
      (SELECT count(*)
FROM  (SELECT user_id,
              min(date) as first_order_date
       FROM   info
       WHERE  order_id not in (SELECT order_id
                               FROM   info
                               WHERE  action = 'cancel_order')
       GROUP BY user_id) as first_orders
WHERE  first_order_date = t1.date) as first_orders
FROM   info t1
GROUP BY date
ORDER BY date
```

### 17. Определите 10 самых популярных товаров в таблице `orders`. Самыми популярными товарами будем считать те, которые встречались в заказах чаще всего. Если товар встречается в одном заказе несколько раз (когда было куплено несколько единиц товара), это тоже учитывается при подсчёте. Учитывайте только неотменённые заказы.

```sql
SELECT product_id,
       times_purchased
FROM   (SELECT unnest(product_ids) as product_id,
               count(*) as times_purchased
        FROM   orders
        WHERE  order_id not in (SELECT order_id
                                FROM   user_actions
                                WHERE  action = 'cancel_order')
        GROUP BY product_id
        ORDER BY times_purchased desc limit 10) t1
ORDER BY product_id
```

### 18. Из таблицы `orders` выведите id и содержимое заказов, которые включают хотя бы один из пяти самых дорогих товаров, доступных в нашем сервисе.



```sql
WITH top_products as (
    SELECT product_id
    FROM   products
    ORDER BY price desc limit 5),
unnest as (
    SELECT order_id,
           product_ids,
           unnest(product_ids) as product_id
    FROM   orders
)
SELECT DISTINCT order_id,
                product_ids
FROM   unnest
WHERE  product_id in (SELECT *
                      FROM   top_products)
ORDER BY order_id
```